In [248]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.pipeline import Pipeline
from Balancing_data import undersampling, oversampling
from collections import Counter
from  imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer

In [249]:
train_cosine = pd.read_csv('train_test_data/train_unbalanced_cosine.csv')
test_cosine = pd.read_csv('train_test_data/test_unbalanced_cosine.csv')

train_bert = pd.read_csv('train_test_data/train_unbalanced_bert.csv')
test_bert = pd.read_csv('train_test_data/test_unbalanced_bert.csv')

train_raw = pd.read_csv('train_test_data/train_unbalanced_prep.csv').drop(['Unnamed: 0'], axis=1)
test_raw = pd.read_csv('train_test_data/test_unbalanced_prep.csv').drop(['Unnamed: 0'], axis=1)

In [250]:
cosine_da = train_cosine.drop(['URL', 'Cosine_DS_LastWorkDesc', 'Cosine_DS_LastWorkDesc2',	'Cosine_DS_Description', 'DataScientist'], axis=1)
cosine_ds = train_cosine.drop(['URL', 'Cosine_DA_LastWorkDesc', 'Cosine_DA_LastWorkDesc2',	'Cosine_DA_Description', 'DataAnalyst'], axis=1)

In [251]:
train_cosine

,ExpPeriod,Salary,Age,Gender,WorkType,WorkSchedule,N_places,Top 10 work,N_changeWork,N_Langs,...,top_city,EduLevelCat,Cosine_DA_LastWorkDesc,Cosine_DA_LastWorkDesc2,Cosine_DA_Description,Cosine_DS_LastWorkDesc,Cosine_DS_LastWorkDesc2,Cosine_DS_Description,DataAnalyst,DataScientist
0,0.899332,-0.032825,0.471765,0.0,1.0,0.0,1.344351,0.0,-0.228412,-0.290421,...,1.0,1.0,0.020599,0.009887,0.032961,0.017386,0.004344,0.039497,0.0,0.0
1,-0.613145,-0.032825,-0.851000,0.0,1.0,0.0,1.013097,0.0,-0.228412,0.933674,...,1.0,1.0,0.112261,0.007916,0.004854,0.109529,0.000000,0.003678,0.0,0.0
2,-1.324899,-0.032825,-1.071461,1.0,1.0,0.0,-1.305683,0.0,-0.228412,0.933674,...,1.0,1.0,0.099322,0.000000,0.006594,0.128600,0.000000,0.019851,0.0,0.0
3,0.098609,-0.032825,-0.263104,0.0,1.0,0.0,0.019334,0.0,-0.228412,-0.290421,...,1.0,1.0,0.007563,0.012057,0.000000,0.003404,0.007315,0.013863,0.0,0.0
4,-0.247383,-0.032825,-0.263104,3.0,1.0,0.0,2.338114,0.0,3.064879,3.381864,...,1.0,1.0,0.015857,0.012545,0.039033,0.015414,0.006676,0.049572,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7166,1.502346,-0.032825,0.839200,0.0,1.0,0.0,0.681843,0.0,-0.228412,-0.290421,...,1.0,1.0,0.011104,0.015722,0.012739,0.008316,0.008928,0.027551,0.0,0.0
7167,-0.247383,-0.032825,0.912687,1.0,1.0,0.0,-0.643175,1.0,-0.228412,-1.514516,...,1.0,0.0,0.013936,0.015864,0.000000,0.009933,0.007549,0.000000,0.0,0.0
7168,-0.010131,-0.032825,-0.042644,2.0,1.0,0.0,0.019334,0.0,-0.228412,-0.290421,...,0.0,1.0,0.004725,0.042903,0.016787,0.010510,0.031528,0.026345,0.0,0.0
7169,0.681852,-0.032825,1.133148,1.0,0.0,0.0,-0.311920,0.0,-0.228412,-0.290421,...,1.0,1.0,0.000000,0.007769,0.014185,0.002102,0.008927,0.015454,0.0,0.0


In [252]:
X_under_cosine_da, y_under_cosine_da = undersampling(cosine_da.drop(['DataAnalyst'], axis=1), cosine_da['DataAnalyst'])
X_under_cosine_ds, y_under_cosine_ds = undersampling(cosine_ds.drop(['DataScientist'], axis=1), cosine_ds['DataScientist'])

X_test_cosine_da, y_test_cosine_da = test_cosine.drop(['DataAnalyst', 'DataScientist', 'URL', 'Cosine_DS_LastWorkDesc', 'Cosine_DS_LastWorkDesc2',	'Cosine_DS_Description'], axis=1), test_cosine['DataAnalyst']
X_test_cosine_ds, y_test_cosine_ds = test_cosine.drop(['DataAnalyst', 'DataScientist', 'URL', 'Cosine_DA_LastWorkDesc', 'Cosine_DA_LastWorkDesc2',	'Cosine_DA_Description'], axis=1), test_cosine['DataScientist']

In [253]:
bert_da = train_bert.drop(['URL', 'ds', 'DataScientist', 'Description', 'LastWorkDesc', 'LastWorkDesc2'], axis=1)
bert_ds = train_bert.drop(['URL', 'da', 'DataAnalyst', 'Description', 'LastWorkDesc', 'LastWorkDesc2'], axis=1)

In [254]:
X_under_bert_da, y_under_bert_da = undersampling(bert_da.drop(['DataAnalyst'], axis=1), bert_da['DataAnalyst'])
X_under_bert_ds, y_under_bert_ds = undersampling(bert_ds.drop(['DataScientist'], axis=1), bert_ds['DataScientist'])

X_test_bert_da, y_test_bert_da = test_bert.drop(['DataAnalyst', 'DataScientist', 'URL', 'ds', 'Description', 'LastWorkDesc', 'LastWorkDesc2'], axis=1), test_bert['DataAnalyst']
X_test_bert_ds, y_test_bert_ds = test_bert.drop(['DataAnalyst', 'DataScientist', 'URL', 'da', 'Description', 'LastWorkDesc', 'LastWorkDesc2'], axis=1), test_bert['DataScientist']

In [274]:
X_over_cosine_da, y_over_cosine_da = oversampling(cosine_da.drop(['DataAnalyst'], axis=1), cosine_da['DataAnalyst'])
X_over_cosine_ds, y_over_cosine_ds = oversampling(cosine_ds.drop(['DataScientist'], axis=1), cosine_ds['DataScientist'])

In [278]:
X_over_bert_da, y_over_bert_da = oversampling(bert_da.drop(['DataAnalyst'], axis=1), bert_da['DataAnalyst'])
X_over_bert_ds, y_over_bert_ds = oversampling(bert_ds.drop(['DataScientist'], axis=1), bert_ds['DataScientist'])

# Models

In [265]:
def xgboost_clf(X_train, y_train, X_test, y_test, type_balance, type_scoring, prof):
    model = XGBClassifier().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(prof)
    print(type_balance, 'and', type_scoring)
    print('Accuracy score is: ', accuracy_score(y_pred, y_test))
    print('Precision: ', precision_score(y_pred, y_test))
    print('Recall: ', recall_score(y_pred, y_test))
    print('F1-Score: ', f1_score(y_pred, y_test))

In [266]:
def lr_clf(X_train, y_train, X_test, y_test, type_balance, type_scoring, prof):
    model = LogisticRegression(random_state=42, solver='lbfgs', C=20, max_iter=1000).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(prof)
    print(type_balance, 'and', type_scoring)
    print('Accuracy score is: ', accuracy_score(y_pred, y_test))
    print('Precision: ', precision_score(y_pred, y_test))
    print('Recall: ', recall_score(y_pred, y_test))
    print('F1-Score: ', f1_score(y_pred, y_test))

# Undersampling data experiments

In [267]:
xgboost_clf(X_under_cosine_da, y_under_cosine_da, X_test_cosine_da, y_test_cosine_da, 'Undersampling', 'Cosine', 'DataAnalyst')

DataAnalyst
Undersampling and Cosine
Accuracy score is:  0.7766624843161857
Precision:  0.7848837209302325
Recall:  0.21360759493670886
F1-Score:  0.3358208955223881


In [268]:
lr_clf(X_under_cosine_da, y_under_cosine_da, X_test_cosine_da, y_test_cosine_da, 'Undersampling', 'Cosine', 'DataAnalyst')

DataAnalyst
Undersampling and Cosine
Accuracy score is:  0.8402342116269343
Precision:  0.7151162790697675
Recall:  0.26973684210526316
F1-Score:  0.39171974522292996


In [269]:
xgboost_clf(X_under_cosine_ds, y_under_cosine_ds, X_test_cosine_ds, y_test_cosine_ds, 'Undersampling', 'Cosine', 'DataScientist')

DataScientist
Undersampling and Cosine
Accuracy score is:  0.876620660811376
Precision:  0.8928571428571429
Recall:  0.07886435331230283
F1-Score:  0.14492753623188406


In [270]:
lr_clf(X_under_cosine_ds, y_under_cosine_ds, X_test_cosine_ds, y_test_cosine_ds, 'Undersampling', 'Cosine', 'DataScientist')

DataScientist
Undersampling and Cosine
Accuracy score is:  0.8724383103304056
Precision:  0.8928571428571429
Recall:  0.0764525993883792
F1-Score:  0.1408450704225352


In [271]:
xgboost_clf(X_under_bert_da, y_under_bert_da, X_test_bert_da, y_test_bert_da, 'Undersampling', 'BERT', 'DataAnalyst')

DataAnalyst
Undersampling and BERT
Accuracy score is:  0.7909992372234935
Precision:  0.6914285714285714
Recall:  0.1967479674796748
F1-Score:  0.30632911392405066


In [272]:
lr_clf(X_under_bert_da, y_under_bert_da, X_test_bert_da, y_test_bert_da, 'Undersampling', 'BERT', 'DataAnalyst')

DataAnalyst
Undersampling and BERT
Accuracy score is:  0.8405797101449275
Precision:  0.6285714285714286
Recall:  0.23758099352051837
F1-Score:  0.3448275862068965


# Oversampling data experiments

In [279]:
xgboost_clf(X_over_cosine_da, y_over_cosine_da, X_test_cosine_da, y_test_cosine_da, 'Oversampling', 'Cosine', 'DataAnalyst')

DataAnalyst
Oversampling and Cosine
Accuracy score is:  0.9330823923044751
Precision:  0.3313953488372093
Recall:  0.5588235294117647
F1-Score:  0.416058394160584


In [280]:
xgboost_clf(X_over_cosine_ds, y_over_cosine_ds, X_test_cosine_ds, y_test_cosine_ds, 'Oversampling', 'Cosine', 'DataScientist')

DataScientist
Oversampling and Cosine
Accuracy score is:  0.9828523630280217
Precision:  0.25
Recall:  0.25925925925925924
F1-Score:  0.2545454545454545


In [282]:
xgboost_clf(X_over_bert_da, y_over_bert_da, X_test_bert_da, y_test_bert_da, 'Oversampling', 'BERT', 'DataAnalyst')

DataAnalyst
Oversampling and BERT
Accuracy score is:  0.9355453852021358
Precision:  0.32
Recall:  0.5283018867924528
F1-Score:  0.39857651245551606


In [281]:
xgboost_clf(X_over_bert_ds, y_over_bert_ds, X_test_bert_ds, y_test_bert_ds, 'Oversampling', 'BERT', 'DataScientist')

DataScientist
Oversampling and BERT
Accuracy score is:  0.986651411136537
Precision:  0.21428571428571427
Recall:  0.3157894736842105
F1-Score:  0.2553191489361702


In [283]:
lr_clf(X_over_cosine_da, y_over_cosine_da, X_test_cosine_da, y_test_cosine_da, 'Oversampling', 'Cosine', 'DataAnalyst')

DataAnalyst
Oversampling and Cosine
Accuracy score is:  0.8419071518193224
Precision:  0.7383720930232558
Recall:  0.27608695652173915
F1-Score:  0.40189873417721517


In [284]:
lr_clf(X_over_cosine_ds, y_over_cosine_ds, X_test_cosine_ds, y_test_cosine_ds, 'Oversampling', 'Cosine', 'DataScientist')

DataScientist
Oversampling and Cosine
Accuracy score is:  0.9355918025930573
Precision:  0.9285714285714286
Recall:  0.14606741573033707
F1-Score:  0.2524271844660194


In [285]:
lr_clf(X_over_bert_da, y_over_bert_da, X_test_bert_da, y_test_bert_da, 'Oversampling', 'BERT', 'DataAnalyst')

DataAnalyst
Oversampling and BERT
Accuracy score is:  0.8447749809305873
Precision:  0.6571428571428571
Recall:  0.24891774891774893
F1-Score:  0.36106750392464676


In [286]:
lr_clf(X_over_bert_ds, y_over_bert_ds, X_test_bert_ds, y_test_bert_ds, 'Oversampling', 'BERT', 'DataScientist')

DataScientist
Oversampling and BERT
Accuracy score is:  0.944698703279939
Precision:  0.8571428571428571
Recall:  0.14545454545454545
F1-Score:  0.24870466321243523
